In [ ]:
import numpy as np
import pandas as pd
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 25
pd.options.display.max_columns = 20
pd.options.display.max_colwidth = 82
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc("figure", figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

在数据分析的过程中，相当多的时间要用在数据准备上：加载、清理、转换以及重塑。这些工作会占到数据分析时间的80%或更多。许多研究者都选择使用通用编程语言（如Python、Perl、R或Java）或UNIX文本处理工具（如sed或awk）对数据格式进行专门处理。幸运的是，pandas和内置的Python标准库提供了一组高级、灵活且快速的工具，可以让你轻松地将数据规整为想要的格式。

在本章节中，我们会讨论处理缺失数据、重复数据、字符串操作和其它分析数据转换的工具。

## 处理缺失数据
在数据分析过程中，缺失数据是经常发生的。pandas的目标之一就是尽量轻松地处理缺失数据。例如，pandas对象的所有描述性统计默认都不包括缺失数据。

缺失数据在pandas中呈现的方式虽然有些不完美，但对于大多数用户可以保证功能正常。对于数值数据，pandas使用浮点值NaN（Not a Number）表示缺失数据，可以很方便的检测出来：

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
float_data = pd.Series([1.2, -3.5, np.nan, 0])
float_data

In [ ]:
float_data.isna()

In [ ]:
string_data = pd.Series(["aardvark", np.nan, None, "avocado"])
string_data
string_data.isna()

pandas项目中还在不断优化内部细节以更好处理缺失数据，像用户API功能，例如pandas.isnull，去除了许多恼人的细节。下表列出了一些关于缺失数据处理的函数。
![pandas_data_clean](./figures/pandas_data_clean.jpg)

### 滤除缺失数据
过滤掉缺失数据的办法有很多种。你可以通过pandas.isnull或布尔索引的手工方法，但dropna可能会更实用一些。对于一个Series，dropna返回一个仅含非空数据和索引值的Series：

In [ ]:
data = pd.Series([1, np.nan, 3.5, np.nan, 7])
data

In [ ]:
data.dropna()

In [ ]:
# 这等价于：
data[data.notna()]

而对于DataFrame对象，事情就有点复杂了。你可能希望丢弃全NaN或含有NaN的行或列。dropna默认丢弃任何含有缺失值的行：

In [ ]:
data = pd.DataFrame([[1., 6.5, 3.], [1., np.nan, np.nan],
                     [np.nan, np.nan, np.nan], [np.nan, 6.5, 3.]])
data

In [ ]:
data.dropna()

传入how='all'将只丢弃全为NaN的那些行：

In [ ]:
data.dropna(how="all")

用这种方式丢弃列，只需传入axis=1即可：

In [ ]:
data[4] = np.nan
data

In [ ]:
data.dropna(axis="columns", how="all")

另一个滤除DataFrame行的问题涉及时间序列数据。假设你只想留下一部分观测数据，可以用thresh参数实现此目的：

In [ ]:
df = pd.DataFrame(np.random.standard_normal((7, 3)))
df.iloc[:4, 1] = np.nan
df.iloc[:2, 2] = np.nan
df

In [ ]:
df.dropna()

In [ ]:
df.dropna(thresh=2)

### 填充缺失数据
你可能不想滤除缺失数据（有可能会丢弃跟它有关的其他数据），而是希望通过其他方式填补那些“空洞”。对于大多数情况而言，fillna方法是最主要的函数。通过一个常数调用fillna就会将缺失值替换为那个常数值：

In [ ]:
df.fillna(0)

若是通过一个字典调用fillna，就可以实现对不同的列填充不同的值：

In [ ]:
df.fillna({1: 0.5, 2: 0})

对reindexing有效的那些插值方法也可用于fillna：

In [ ]:
df = pd.DataFrame(np.random.standard_normal((6, 3)))
df.iloc[2:, 1] = np.nan
df.iloc[4:, 2] = np.nan
df

In [ ]:
df.fillna(method="ffill")

In [ ]:
df.fillna(method="ffill", limit=2)

你还可以利用fillna实现更多别的功能。比如说，你可以传入Series的平均值或中位数：

In [ ]:
data = pd.Series([1., np.nan, 3.5, np.nan, 7])
data

In [ ]:
data.fillna(data.mean())

## 数据转换
本章到目前为止介绍的都是数据的重排。另一类重要操作则是过滤、清理以及其他的转换工作。

### 移除重复数据
DataFrame中出现重复行有多种原因。下面就是一个例子：

In [ ]:
data = pd.DataFrame({"k1": ["one", "two"] * 3 + ["two"],
                     "k2": [1, 1, 2, 3, 3, 4, 4]})
data

DataFrame的duplicated方法返回一个布尔型Series，表示各行是否是重复行（前面出现过的行）：

In [ ]:
data.duplicated()

还有一个与此相关的drop_duplicates方法，它会返回一个DataFrame，重复的数组会标为False：

In [ ]:
data.drop_duplicates()

这两个方法默认会判断全部列，你也可以指定部分列进行重复项判断。假设我们还有一列值，且只希望根据k1列过滤重复项：

In [ ]:
data["v1"] = range(7)
data

In [ ]:
data.drop_duplicates(subset=["k1"])

duplicated和drop_duplicates默认保留的是第一个出现的值组合。传入keep='last'则保留最后一个：

In [ ]:
data.drop_duplicates(["k1", "k2"], keep="last")

### 利用函数或映射进行数据转换
对于许多数据集，你可能希望根据数组、Series或DataFrame列中的值来实现转换工作。我们来看看下面这组有关肉类的数据：

In [ ]:
data = pd.DataFrame({"food": ["bacon", "pulled pork", "bacon",
                              "pastrami", "corned beef", "bacon",
                              "pastrami", "honey ham", "nova lox"],
                     "ounces": [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

假设你想要添加一列表示该肉类食物来源的动物类型。我们先编写一个不同肉类到动物的映射：

In [ ]:
meat_to_animal = {
  "bacon": "pig",
  "pulled pork": "pig",
  "pastrami": "cow",
  "corned beef": "cow",
  "honey ham": "pig",
  "nova lox": "salmon"
}

Series的map方法可以接受一个函数或含有映射关系的字典型对象：

In [ ]:
data["animal"] = data["food"].map(meat_to_animal)
data

我们也可以传入一个能够完成全部这些工作的函数：

In [ ]:
def get_animal(x):
    return meat_to_animal[x]
    
data["food"].map(get_animal)
data

使用map是一种实现元素级转换以及其他数据清理工作的便捷方式。

### 替换值
利用fillna方法填充缺失数据可以看做值替换的一种特殊情况。前面已经看到，map可用于修改对象的数据子集，而replace则提供了一种实现该功能的更简单、更灵活的方式。我们来看看下面这个Series：

In [ ]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

-999这个值可能是一个表示缺失数据的标记值。要将其替换为pandas能够理解的NaN值，我们可以利用replace来产生一个新的Series（除非传入inplace=True）:

In [ ]:
data.replace(-999, np.nan)

如果你希望一次性替换多个值，可以传入一个由待替换值组成的列表以及一个替换值：

In [ ]:
data.replace([-999, -1000], np.nan)

要让每个值有不同的替换值，可以传递一个替换列表：

In [ ]:
data.replace([-999, -1000], [np.nan, 0])

传入的参数也可以是字典：

In [ ]:
data.replace({-999: np.nan, -1000: 0})

- 笔记：data.replace方法与data.str.replace不同，后者做的是字符串的元素级替换。我们会在后面学习Series的字符串方法。

### 重命名轴索引
跟Series中的值一样，轴标签也可以通过函数或映射进行转换，从而得到一个新的不同标签的对象。轴还可以被就地修改，而无需新建一个数据结构。接下来看看这个简单的例子：

In [ ]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=["Ohio", "Colorado", "New York"],
                    columns=["one", "two", "three", "four"])
data

跟Series一样，轴索引也有一个map方法：

In [ ]:
def transform(x):
    return x[:4].upper()

data.index.map(transform)

你可以将其赋值给index，这样就可以对DataFrame进行就地修改：

In [ ]:
data.index = data.index.map(transform)
data

如果想要创建数据集的转换版（而不是修改原始数据），比较实用的方法是rename：

In [ ]:
data.rename(index=str.title, columns=str.upper)

特别说明一下，rename可以结合字典型对象实现对部分轴标签的更新：

In [ ]:
data.rename(index={"OHIO": "INDIANA"},
            columns={"three": "peekaboo"})

### 离散化和面元划分
为了便于分析，连续数据常常被离散化或拆分为“面元”（bin）。假设有一组人员数据，而你希望将它们划分为不同的年龄组：

In [ ]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

接下来将这些数据划分为“18到25”、“26到35”、“35到60”以及“60以上”几个面元。要实现该功能，你需要使用pandas的cut函数：

In [ ]:
bins = [18, 25, 35, 60, 100]
age_categories = pd.cut(ages, bins)
age_categories

pandas返回的是一个特殊的Categorical对象。结果展示了pandas.cut划分的面元。你可以将其看做一组表示面元名称的字符串。它的底层含有一个表示不同分类名称的类型数组，以及一个codes属性中的年龄数据的标签：

In [ ]:
age_categories.codes

In [ ]:
age_categories.categories

In [ ]:
age_categories.categories[0]

In [ ]:
pd.value_counts(age_categories)

pd.value_counts(cats)是pandas.cut结果的面元计数。

跟“区间”的数学符号一样，圆括号表示开端，而方括号则表示闭端（包括）。哪边是闭端可以通过right=False进行修改：

In [ ]:
pd.cut(ages, bins, right=False)

你可以通过传递一个列表或数组到labels，设置自己的面元名称：

In [ ]:
group_names = ["Youth", "YoungAdult", "MiddleAged", "Senior"]
pd.cut(ages, bins, labels=group_names)

如果向cut传入的是面元的数量而不是确切的面元边界，则它会根据数据的最小值和最大值计算等长面元。下面这个例子中，我们将一些均匀分布的数据分成四组：

In [ ]:
data = np.random.uniform(size=20)
data

In [ ]:
pd.cut(data, 4, precision=2)

qcut是一个非常类似于cut的函数，它可以根据样本分位数对数据进行面元划分。根据数据的分布情况，cut可能无法使各个面元中含有相同数量的数据点。而qcut由于使用的是样本分位数，因此可以得到大小基本相等的面元：

In [ ]:
data = np.random.standard_normal(1000)
quartiles = pd.qcut(data, 4, precision=2)
quartiles

In [ ]:
pd.value_counts(quartiles)

与cut类似，你也可以传递自定义的分位数（0到1之间的数值，包含端点）：

In [ ]:
pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.]).value_counts()

本章稍后在讲解聚合和分组运算时会再次用到cut和qcut，因为这两个离散化函数对分位和分组分析非常重要。

### 检测和过滤异常值
过滤或变换异常值（outlier）在很大程度上就是运用数组运算。来看一个含有正态分布数据的DataFrame：

In [ ]:
data = pd.DataFrame(np.random.randn(1000, 4))
data.describe()

假设你想要找出某列中绝对值大小超过3的值：

In [ ]:
col = data[2]
col[col.abs() > 3]

要选出全部含有“超过3或－3的值”的行，你可以在布尔型DataFrame中使用any方法：

In [ ]:
data[(data.abs() > 3).any(axis="columns")]

根据这些条件，就可以对值进行设置。下面的代码可以将值限制在区间－3到3以内：

In [ ]:
data[data.abs() > 3] = np.sign(data) * 3
data.describe()

根据数据的值是正还是负，np.sign(data)可以生成1和-1：

In [ ]:
np.sign(data).head()

### 排列和随机采样
利用numpy.random.permutation函数可以轻松实现对Series或DataFrame的列的排列工作（permuting，随机重排序）。通过需要排列的轴的长度调用permutation，可产生一个表示新顺序的整数数组：

In [ ]:
df = pd.DataFrame(np.arange(5 * 7).reshape((5, 7)))
df

In [ ]:
sampler = np.random.permutation(5)
sampler

然后就可以在基于iloc的索引操作或take函数中使用该数组了：

In [ ]:
df.take(sampler)

In [ ]:
df.iloc[sampler]

In [ ]:
column_sampler = np.random.permutation(7)
column_sampler
df.take(column_sampler, axis="columns")

如果不想用替换的方式选取随机子集，可以在Series和DataFrame上使用sample方法：

In [ ]:
df.sample(n=3)

要通过替换的方式产生样本（允许重复选择），可以传递replace=True到sample：

In [ ]:
choices = pd.Series([5, 7, -1, 6, 4])
choices.sample(n=10, replace=True)

### 计算指标/哑变量
另一种常用于统计建模或机器学习的转换方式是：将分类变量（categorical variable）转换为“哑变量”或“指标矩阵”。

如果DataFrame的某一列中含有k个不同的值，则可以派生出一个k列矩阵或DataFrame（其值全为1和0）。pandas有一个get_dummies函数可以实现该功能（其实自己动手做一个也不难）。使用之前的一个DataFrame例子：

In [ ]:
df = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "b"],
                   "data1": range(6)})
df

In [ ]:
pd.get_dummies(df["key"], dtype=float)

有时候，你可能想给指标DataFrame的列加上一个前缀，以便能够跟其他数据进行合并。get_dummies的prefix参数可以实现该功能：

In [ ]:
dummies = pd.get_dummies(df["key"], prefix="key", dtype=float)
df_with_dummy = df[["data1"]].join(dummies)
df_with_dummy

## 拓展数据类型
NumPy 是一个主要用于处理数值数据的数组计算库。许多 pandas 概念（如缺失数据）都是利用 NumPy 中可用的功能实现的，同时努力最大化 NumPy 和 pandas 库之间的兼容性。

在 NumPy 的基础上开发有许多缺陷，例如：

* 某些数字数据类型（如整数和布尔型）的缺失数据处理不完整。因此，当在此类数据中引入缺失数据时，pandas 会将数据类型转换为 float64，并使用 np.nan 表示空值。这在许多 pandas 算法中会带来一些小问题。

* 包含大量字符串数据的数据集计算成本很高，并占用大量内存。

* 如果不使用计算成本高昂的 Python 对象数组，就无法高效地支持某些数据类型，如时间间隔、timedeltas 和带有时区的时间戳。

最近，pandas 开发了一个扩展类型系统，允许添加新的数据类型，即使 NumPy 本身不支持这些类型。这些新数据类型可以与来自 NumPy 数组的数据一起被视为第一类数据。

下面我们来看一个例子，创建一个缺失值的整数系列：

In [ ]:
s = pd.Series([1, 2, 3, None])
s

In [ ]:
s.dtype

主要出于向后兼容的原因，Series 使用了传统的行为，即使用 float64 数据类型和 np.nan 来表示缺失值。我们可以使用 pandas.Int64Dtype 来创建这个 Series：

In [ ]:
s = pd.Series([1, 2, 3, None], dtype=pd.Int64Dtype())
s

In [ ]:
s.isna()

In [ ]:
s.dtype

输出 NA 表示扩展类型数组缺少一个值。这使用了特殊的 pandas.NA 哨兵值：

In [ ]:
s[3]

In [ ]:
s[3] is pd.NA

我们也可以使用简写 “Int64 ”来指定类型，而不是 pd.Int64Dtype()。大写是必要的，否则它将是一个基于 NumPy 的非扩展类型：

In [ ]:
s = pd.Series([1, 2, 3, None], dtype="Int64")

pandas 还有一个专门用于字符串数据的扩展类型，它不使用 NumPy 对象数组（它需要 pyarrow 库，你可能需要单独安装）：

In [ ]:
s = pd.Series(['one', 'two', None, 'three'], dtype=pd.StringDtype())
s

这些字符串数组通常使用更少的内存，在对大型数据集进行操作时，计算效率往往更高。

另一个重要的扩展类型是分类(Categorical)，我们将在分类数据中对此进行更详细的讨论。

扩展类型可以传递给 Series astype 方法，使您可以在数据清理过程中轻松进行转换：

In [ ]:
df = pd.DataFrame({"A": [1, 2, None, 4],
                   "B": ["one", "two", "three", None],
                   "C": [False, None, False, True]})
df

In [ ]:
df["A"] = df["A"].astype("Int64")
df["B"] = df["B"].astype("string")
df["C"] = df["C"].astype("boolean")
df

## 字符串操作
Python能够成为流行的数据处理语言，部分原因是其简单易用的字符串和文本处理功能。大部分文本运算都直接做成了字符串对象的内置方法。对于更为复杂的模式匹配和文本操作，则可能需要用到正则表达式。pandas对此进行了加强，它使你能够对整组数据应用字符串表达式和正则表达式，而且能处理烦人的缺失数据。

### 字符串对象方法
对于许多字符串处理和脚本应用，内置的字符串方法已经能够满足要求了。例如，以逗号分隔的字符串可以用split拆分成数段：



In [ ]:
val = "a,b,  guido"
val.split(",")

split常常与strip一起使用，以去除空白符（包括换行符）：

In [ ]:
pieces = [x.strip() for x in val.split(",")]
pieces

利用加法，可以将这些子字符串以双冒号分隔符的形式连接起来：

In [ ]:
first, second, third = pieces
first + "::" + second + "::" + third

但这种方式并不是很实用。一种更快更符合Python风格的方式是，向字符串"::"的join方法传入一个列表或元组：

In [ ]:
"::".join(pieces)

其它方法关注的是子串定位。检测子串的最佳方式是利用Python的in关键字，还可以使用index和find：

In [ ]:
"guido" in val

In [ ]:
val.index(",")

In [ ]:
val.find(":") # no :

注意find和index的区别：如果找不到字符串，index将会引发一个异常（而不是返回－1）：

In [ ]:
val.index(":")

与此相关，count可以返回指定子串的出现次数：

In [ ]:
val.count(",")

replace用于将指定模式替换为另一个模式。通过传入空字符串，它也常常用于删除模式：

In [ ]:
val.replace(",", "::")

In [ ]:
val.replace(",", "")

下表列出了Python内置的字符串方法。

这些运算大部分都能使用正则表达式实现（马上就会看到）。

![python_string1](./figures/python_string1.jpg)
![python_string2](./figures/python_string2.jpg)

### 正则表达式
正则表达式提供了一种灵活的在文本中搜索或匹配（通常比前者复杂）字符串模式的方式。正则表达式，常称作regex，是根据正则表达式语言编写的字符串。Python内置的re模块负责对字符串应用正则表达式。我将通过一些例子说明其使用方法。

re模块的函数可以分为三个大类：模式匹配、替换以及拆分。当然，它们之间是相辅相成的。一个regex描述了需要在文本中定位的一个模式，它可以用于许多目的。我们先来看一个简单的例子：假设我想要拆分一个字符串，分隔符为数量不定的一组空白符（制表符、空格、换行符等）。描述一个或多个空白符的正则表达式是\s+：

In [ ]:
import re
text = "foo    bar\t baz  \tqux"
re.split(r"\s+", text)

调用re.split('\s+',text)时，正则表达式会先被编译，然后再在text上调用其split方法。你可以用re.compile自己编译regex以得到一个可重用的regex对象：

In [ ]:
regex = re.compile(r"\s+")
regex.split(text)

如果只希望得到匹配regex的所有模式，则可以使用findall方法：

In [ ]:
regex.findall(text)

* 笔记：如果想避免正则表达式中不需要的转义（\），则可以使用原始字符串字面量如r'C:\x'（也可以编写其等价式'C:\x'）。
* r前缀用于表示原始字符串（raw string）。原始字符串不会对反斜杠进行转义处理。Python 解释器会将 \ 视为普通字符，而不是转义字符。例如r".\name"不会把\n视为换行符。

如果打算对许多字符串应用同一条正则表达式，建议通过re.compile创建regex对象。这样将可以节省大量的CPU时间。

match和search跟findall功能类似。findall返回的是字符串中所有的匹配项，而search则只返回第一个匹配项。match更加严格，它只匹配字符串的首部。来看一个小例子，假设我们有一段文本以及一条能够识别大部分电子邮件地址的正则表达式：

In [ ]:
text = """Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com"""
pattern = r"[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}"

# re.IGNORECASE makes the regex case insensitive
regex = re.compile(pattern, flags=re.IGNORECASE)

* "r"：这表示一个原始字符串的开头。
* [A-Z0-9._%+-]+：匹配电子邮件地址的用户名部分。[...] 表示一个字符集合，+ 表示前面的字符集合至少出现一次，A-Z 匹配大写字母，0-9 匹配数字，.、_、%、+、- 这些字符直接匹配对应的字符。
* @：匹配电子邮件地址中的 @ 符号。
* [A-Z0-9.-]+：匹配电子邮件地址的域名部分。与前面类似，包括大写字母、数字、.、-。
* \.：匹配电子邮件地址中的 . 符号。需要用 \ 转义，因为 . 在正则表达式中表示匹配任意字符。
* [A-Z]{2,4}：匹配电子邮件地址的顶级域名部分。{2,4} 表示前面的字符集合必须重复出现 2 到 4 次，[A-Z] 匹配大写字母。
  
因此，这个正则表达式用来匹配大部分电子邮件地址的模式，但并不是完全准确的，因为电子邮件地址有很多变种和特例。

对text使用findall将得到一组电子邮件地址：

In [ ]:
regex.findall(text)

search返回的是文本中第一个电子邮件地址（以特殊的匹配项对象形式返回）。对于上面那个regex，匹配项对象只能告诉我们模式在原字符串中的起始和结束位置：

In [ ]:
m = regex.search(text)
m

In [ ]:
text[m.start():m.end()]

regex.match则将返回None，因为它只匹配出现在字符串开头的模式：

In [ ]:
print(regex.match(text))

相关的，sub方法可以将匹配到的模式替换为指定字符串，并返回所得到的新字符串：

In [ ]:
print(regex.sub("REDACTED", text))

假设你不仅想要找出电子邮件地址，还想将各个地址分成3个部分：用户名、域名以及域后缀。要实现此功能，只需将待分段的模式的各部分用圆括号包起来即可：

In [ ]:
pattern = r"([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})"
regex = re.compile(pattern, flags=re.IGNORECASE)

由这种修改过的正则表达式所产生的匹配项对象，可以通过其groups方法返回一个由模式各段组成的元组：

In [ ]:
m = regex.match("wesm@bright.net")
m.groups()

对于带有分组功能的模式，findall会返回一个元组列表：

In [ ]:
regex.findall(text)

sub还能通过诸如\1、\2之类的特殊符号访问各匹配项中的分组。符号\1对应第一个匹配的组，\2对应第二个匹配的组，以此类推：

In [ ]:
print(regex.sub(r"Username: \1, Domain: \2, Suffix: \3", text))

Python中还有许多的正则表达式，但大部分都超出了本章节的范围。下表是一个简要概括。
![python_regex](./figures/python_regex.jpg)

### pandas的矢量化字符串函数
清理待分析的散乱数据时，常常需要做一些字符串规整化工作。更为复杂的情况是，含有字符串的列有时还含有缺失数据：

In [ ]:
data = {"Dave": "dave@google.com", "Steve": "steve@gmail.com",
        "Rob": "rob@gmail.com", "Wes": np.nan}
data = pd.Series(data)
data

In [ ]:
data.isna()

通过data.map，所有字符串和正则表达式方法都能被应用于（传入lambda表达式或其他函数）各个值，但是如果存在NA（null）就会报错。为了解决这个问题，Series有一些能够跳过NA值的面向数组方法，进行字符串操作。通过Series的str属性即可访问这些方法。例如，我们可以通过str.contains检查各个电子邮件地址是否含有"gmail"：

In [ ]:
data.str.contains("gmail")

也可以使用正则表达式，还可以加上任意re选项（如IGNORECASE）：

In [ ]:
pattern = r"([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})"
data.str.findall(pattern, flags=re.IGNORECASE)

有两个办法可以实现矢量化的元素获取操作：要么使用str.get，要么在str属性上使用索引：

In [ ]:
matches = data.str.findall(pattern, flags=re.IGNORECASE).str[0]
matches

In [ ]:
matches.str.get(1)

In [ ]:
data.str[:5]

str.extract方法会将获取的组表达为DataFrame格式：

In [ ]:
data.str.extract(pattern, flags=re.IGNORECASE)

下表介绍了更多的pandas字符串方法。
![pandas_string](./figures/pandas_string.jpg)

### This is the End! ：）